# Exercise 2: Function Implementations
## Correlation Matrix Adjustment Methods

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from scipy.linalg import eigh, norm
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## a) Spectral Decomposition Method

This function adjusts a correlation matrix by:
1. Computing eigenvalues and eigenvectors
2. Replacing negative eigenvalues with a small positive value (ε)
3. Reconstructing the matrix
4. Scaling to ensure ones on the diagonal

In [ ]:
def spectral_decomposition(corr_matrix, epsilon=1e-8):
    """
    Adjust correlation matrix using Spectral Decomposition Method.
    
    Parameters:
    -----------
    corr_matrix : array-like
        Estimator of correlation matrix
    epsilon : float, default=1e-8
        Small parameter ε > 0
    
    Returns:
    --------
    adj_corr_matrix : numpy.ndarray
        Adjusted correlation matrix with ones on diagonal
    """
    # Convert to numpy array if needed
    if isinstance(corr_matrix, pd.DataFrame):
        corr_matrix = corr_matrix.values
    
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eigh(corr_matrix)
    
    # Adjust negative eigenvalues
    eigenvalues_adj = np.where(eigenvalues < epsilon, epsilon, eigenvalues)
    
    # Reconstruct matrix
    adj_corr_matrix = eigenvectors @ np.diag(eigenvalues_adj) @ eigenvectors.T
    
    # Scale to get ones on diagonal
    sqrt_diag = np.sqrt(np.diag(adj_corr_matrix))
    adj_corr_matrix = adj_corr_matrix / sqrt_diag[:, None] / sqrt_diag[None, :]
    
    return adj_corr_matrix

## b) Alternating Projection Method

This function uses Dykstra's alternating projection algorithm:
1. Projects onto the set of positive semi-definite matrices
2. Projects onto the set of matrices with unit diagonal
3. Iterates until convergence

In [ ]:
def alternating_projection(corr_matrix, tolerance=1e-12, tau=1e-8):
    """
    Adjust correlation matrix using Alternating Projection Method.
    
    Parameters:
    -----------
    corr_matrix : array-like
        Estimator of correlation matrix
    tolerance : float, default=1e-12
        Tolerance value for convergence
    tau : float, default=1e-8
        τ parameter for minimum eigenvalue threshold
    
    Returns:
    --------
    adj_corr_matrix : numpy.ndarray
        Adjusted correlation matrix
    iterations : int
        Number of iterations needed
    distance : float
        Distance between original and adjusted matrices
    """
    # Convert to numpy array if needed
    if isinstance(corr_matrix, pd.DataFrame):
        corr_matrix = corr_matrix.values
    
    # Step 0: Initialization
    Y = corr_matrix.copy()
    delta_S = 0
    k = 1
    b = np.ones(corr_matrix.shape[0])
    
    # Iterative Algorithm
    while True:
        # Step 1: Dykstra's correction
        R = Y - delta_S
        
        # Step 2: Projection onto S_n^+
        eigenvals_R, eigenvecs_R = eigh(R)
        eigenvals_R[eigenvals_R < tau] = tau
        X_k = eigenvecs_R @ np.diag(eigenvals_R) @ eigenvecs_R.T
        
        # Step 3
        delta_S = X_k - R
        
        # Step 4: Projection onto U={X ∈ S_n: x_ii=1}
        Y = X_k - np.diag(X_k.diagonal() - b)
        
        # Step 5: Check convergence
        norm_k = norm(X_k.diagonal() - b)
        
        if norm_k <= tolerance:
            break
        else:
            k += 1
            
        # Safety check to prevent infinite loops
        if k > 10000:
            print(f"Warning: Maximum iterations reached. Current norm: {norm_k}")
            break
    
    # Scaling to obtain ones on the main diagonal
    X_k_scalar = np.diag(1/np.sqrt(np.diag(X_k)))
    adj_corr_matrix = X_k_scalar @ X_k @ X_k_scalar
    
    # Calculate distance
    distance = np.linalg.norm(corr_matrix - adj_corr_matrix)
    
    return adj_corr_matrix, k, distance

## Test the Functions

Let's test both functions with a simple non-positive definite correlation matrix.

In [ ]:
# Create a test correlation matrix (not positive definite)
test_corr = np.array([[1.00, 0.99, 0.99],
                      [0.99, 1.00, -0.50],
                      [0.99, -0.50, 1.00]])

print("Test Correlation Matrix:")
print(test_corr)
print("\nOriginal eigenvalues:")
eigenvals_orig, _ = eigh(test_corr)
print(eigenvals_orig)
print(f"\nMinimum eigenvalue: {eigenvals_orig.min():.10f}")
print(f"Is positive definite: {eigenvals_orig.min() > 0}")

In [ ]:
# Test Spectral Decomposition
print("\n" + "="*70)
print("SPECTRAL DECOMPOSITION METHOD")
print("="*70)

corr_spectral = spectral_decomposition(test_corr)
eigenvals_spectral, _ = eigh(corr_spectral)
distance_spectral = np.linalg.norm(test_corr - corr_spectral)

print("\nAdjusted correlation matrix:")
print(corr_spectral)
print("\nAdjusted eigenvalues:")
print(eigenvals_spectral)
print(f"\nMinimum eigenvalue: {eigenvals_spectral.min():.10f}")
print(f"Distance from original: {distance_spectral:.10f}")
print(f"Diagonal elements: {np.diag(corr_spectral)}")

In [ ]:
# Test Alternating Projection
print("\n" + "="*70)
print("ALTERNATING PROJECTION METHOD")
print("="*70)

corr_alt, iterations, distance_alt = alternating_projection(test_corr)
eigenvals_alt, _ = eigh(corr_alt)

print("\nAdjusted correlation matrix:")
print(corr_alt)
print("\nAdjusted eigenvalues:")
print(eigenvals_alt)
print(f"\nMinimum eigenvalue: {eigenvals_alt.min():.10f}")
print(f"Number of iterations: {iterations}")
print(f"Distance from original: {distance_alt:.10f}")
print(f"Diagonal elements: {np.diag(corr_alt)}")

In [ ]:
# Compare both methods
print("\n" + "="*70)
print("COMPARISON")
print("="*70)
print(f"\nSpectral Decomposition distance:  {distance_spectral:.10f}")
print(f"Alternating Projection distance:   {distance_alt:.10f}")
print(f"\nDifference in distances: {abs(distance_spectral - distance_alt):.10f}")
print(f"\nBetter method (smaller distance): ", end="")
if distance_spectral < distance_alt:
    print("Spectral Decomposition")
else:
    print("Alternating Projection")

## Visualizations

In [ ]:
# 1. Eigenvalue comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

methods = ['Original', 'Spectral', 'Alternating']
eigenvals_list = [eigenvals_orig, eigenvals_spectral, eigenvals_alt]
colors = ['red', 'blue', 'green']

for ax, method, eigenvals, color in zip(axes, methods, eigenvals_list, colors):
    ax.bar(range(len(eigenvals)), eigenvals, color=color, alpha=0.7)
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax.set_xlabel('Eigenvalue Index')
    ax.set_ylabel('Eigenvalue')
    ax.set_title(f'{method} Matrix')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Eigenvalue Comparison', y=1.02, fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# 2. Correlation matrix heatmaps
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

matrices = [test_corr, corr_spectral, corr_alt]
titles = ['Original', 'Spectral Decomposition', 'Alternating Projection']

for ax, matrix, title in zip(axes, matrices, titles):
    sns.heatmap(matrix, annot=True, fmt='.3f', cmap='RdBu_r', center=0, 
                vmin=-1, vmax=1, square=True, ax=ax, cbar_kws={'shrink': 0.8})
    ax.set_title(title)

plt.tight_layout()
plt.suptitle('Correlation Matrix Comparison', y=1.02, fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# 3. Distance comparison
fig, ax = plt.subplots(figsize=(8, 5))

methods = ['Spectral\nDecomposition', 'Alternating\nProjection']
distances = [distance_spectral, distance_alt]
colors = ['steelblue', 'coral']

bars = ax.bar(methods, distances, color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('Frobenius Norm Distance', fontsize=12)
ax.set_title('Distance from Original Matrix', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, dist in zip(bars, distances):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{dist:.6f}', ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()

## Summary

Both functions successfully:
- Convert non-positive definite correlation matrices to positive semi-definite ones
- Maintain ones on the diagonal
- Preserve symmetry

**Key differences:**
- **Spectral Decomposition**: Faster, simpler, but may have larger distance from original
- **Alternating Projection**: Iterative, typically provides closer approximation to original matrix